In [1]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate
import jax.numpy as jnp

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=5, suppress=True, linewidth=100)


In [14]:
dt = 0.005
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0,0.0,0.0, 0.0])
u_vec = np.array([0.0]) 
xu_lin_k = jnp.array(np.concatenate((x_init, u_vec)))
x_seq = np.zeros((len_seq, 4))
x_seq[0]= x_init

In [15]:
mjcf_model  = mj_models.create_MJCF_double_pend_dev()
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)

In [16]:

nu = mj_model.nu
nx = mj_model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
mj_model.opt.timestep = dt
mj_data.ctrl = u_vec
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)
print(mj_data.ctrl)

[0. 0.]
[0. 0.]
[0.]


In [17]:
dpend_sys = dyn.double_pend_rel_dyn(g=9.81, m1=m_bar, moi1=moi, d1=h_bar/2, l1=h_bar, 
                                            m2=m_bar, moi2=moi, d2=h_bar/2, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=False)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [18]:
mujoco.mj_forward(mj_model, mj_data)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, xu_lin_k, len(x_init))

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.99984  0.00016  0.005   -0.     ]
 [ 0.00021  0.99943  0.       0.005  ]
 [-0.06295  0.06269  0.99984  0.0001 ]
 [ 0.08375 -0.22996  0.00021  0.99949]]
A dyn: 
 [[ 0.99984  0.00016  0.005    0.     ]
 [ 0.00021  0.99942  0.       0.005  ]
 [-0.06296  0.06271  0.99984  0.00016]
 [ 0.08377 -0.23003  0.00021  0.99942]]
B mujoco: 
 [[ 0.00002]
 [-0.00005]
 [ 0.00854]
 [-0.02132]]
B dyn: 
 [[ 0.00002]
 [-0.00005]
 [ 0.00854]
 [-0.02133]]


In [24]:
x_vec = np.array([1.0, 1.0, 3.0, 0.0])
u_vec = np.array([10.0])
xu_k = jnp.concatenate((x_vec, u_vec))
mj_funcs.set_mj_ctrl_vec(mj_data, u_vec)
mj_funcs.set_mj_state_vec(mj_data, x_vec)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps=1e-6, flg_centered=True)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, xu_k, len(x_vec))

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.99989 -0.00014  0.00505  0.00003]
 [ 0.00027  1.00035 -0.00018  0.00495]
 [-0.04258 -0.05625  1.01956  0.01093]
 [ 0.1089   0.14137 -0.07341  0.98008]]
A dyn: 
 [[ 0.99989 -0.00014  0.00505  0.00003]
 [ 0.00027  1.00035 -0.00019  0.00495]
 [-0.04259 -0.05626  1.01993  0.01089]
 [ 0.10891  0.14141 -0.07438  0.98018]]
B mujoco: 
 [[ 0.00001]
 [-0.00002]
 [ 0.00485]
 [-0.0088 ]]
B dyn: 
 [[ 0.00001]
 [-0.00002]
 [ 0.00448]
 [-0.0082 ]]


In [25]:
pot_energy_seq = np.zeros((len_seq,1))
kin_energy_seq = np.zeros((len_seq,1))
tot_energy_seq = np.zeros((len_seq,1))
pot_energy_seq[0] = dpend_sys.calculate_potential_energy(x_init)
kin_energy_seq[0] = dpend_sys.calculate_kinetic_energy(x_init)
tot_energy_seq[0] = dpend_sys.calculate_total_energy(x_init)